# Register ontology from WebProtégé in Blue Brain Nexus

## Introduction

- This notebook can be used to register / update ontologies from [WebProtégé](https://bbp.epfl.ch/nexus/webprotege/#projects/list) in the Blue Brain Knowledge Graph
- Subsequently, the ontology has to be downloaded from WebProtégé in TTL format
- Ontologies are registered in the neurosciencegraph/datamodels project in the Blue Brain Knowledge Graph

## Install

In [ ]:
# !pip install pyLD
# !pip install ontospy
# !pip install nexusforge

## Import

In [ ]:
import getpass

import rdflib
from rdflib import Namespace
from rdflib.namespace import OWL, RDFS

from ontospy import Ontospy

from kgforge.core import KnowledgeGraphForge

In [ ]:
import bmo.ontologies as bmo 

## Configure forge client

In [ ]:
ENDPOINT = "https://staging.nexus.ocp.bbp.epfl.ch/v1"
BUCKET = "neurosciencegraph/datamodels"

In [ ]:
TOKEN = getpass.getpass()

In [ ]:
forge = KnowledgeGraphForge("https://raw.githubusercontent.com/BlueBrain/nexus-forge/master/examples/notebooks/use-cases/prod-forge-nexus.yml", 
                            endpoint=ENDPOINT, bucket=BUCKET, token=TOKEN)

## Load ontology

In [ ]:
ONTO_PATH = "../../ontologies/bmo.ttl"

Specify here ontology prefix

In [ ]:
PREFIX = "https://neuroshapes.org/"

In [ ]:
ontologyspy = Ontospy(ONTO_PATH, verbose=True)

In [ ]:
for x in ontologyspy.stats(): print(f"{x[0]}: {x[1]}")

In [ ]:
NSG = Namespace('https://neuroshapes.org/')
UBERON = Namespace('http://purl.obolibrary.org/obo/UBERON_')
SKOS = Namespace('http://www.w3.org/2004/02/skos/core#')
OWL = Namespace('http://www.w3.org/2002/07/owl#')

In [ ]:
ontology_graph = rdflib.Graph()
ontology_graph.parse(ONTO_PATH, format="turtle")
ontology_graph.bind('nsg', NSG)
ontology_graph.bind('skos', SKOS)
ontology_graph.bind('owl', OWL)
ontology_graph.bind('UBERON', UBERON)

In [ ]:
ontology = bmo.find_ontology_resource(ontology_graph)

## Preprocess ontology

TODO: Make sure that all the BMO URIs are of the form `{prefix}{labelInCamelCase}`

If ontology does not have a label, add if from the title.

In [ ]:
if not ontology_graph.label(ontology):
    bmo.add_ontology_label(ontology_graph, ontology)

Add relations `defines` from the ontology resource to all the classes.

In [ ]:
bmo.add_defines_relation(ontology_graph, PREFIX)

Convert restrictions on relations between ontology classes to simple (`source_class`, `relation`, `target_class`) triples.

In [ ]:
bmo.restrictions_to_triples(ontology_graph)

Replace webprotege links from `isDefinedBy` relationships with Nexus IDs of respective ontologies

In [ ]:
WEBPROTEGE_TO_NEXUS = {
    # Target ontology ID's to define
    "https://bbp.epfl.ch/nexus/webprotege/#projects/755556fa-73b1-4863-96af-e8359109b4ef/edit/Classes": "https://bbp.epfl.ch/ontologies/lib/molecular-systems",
    "https://bbp.epfl.ch/nexus/webprotege/#projects/facad879-18ea-4499-8f44-b154ed6c0020/edit/Classes": "https://bbp.epfl.ch/ontologies/lib/e-features",
    # Already exist in Nexus
    "https://bbp.epfl.ch/nexus/webprotege/#projects/c0f3a3e7-6dd2-4802-a00a-61ae366a35bb/edit/Classes": "http://bbp.epfl.ch/neurosciencegraph/ontologies/mba",
    "https://bbp.epfl.ch/nexus/webprotege/#projects/7515dc12-ce84-4eea-ba8e-6262670ac741/edit/Classes": "http://bbp.epfl.ch/neurosciencegraph/ontologies/etypes",
    "https://bbp.epfl.ch/nexus/webprotege/#projects/6a23494a-360c-4152-9e81-fd9828f44db9/edit/Classes": "http://bbp.epfl.ch/neurosciencegraph/ontologies/mtypes"
}

In [ ]:
bmo.replace_is_defined_by_uris(ontology_graph, WEBPROTEGE_TO_NEXUS)

## Register ontology

Get remote context from Nexus

In [ ]:
# context = forge.retrieve("https://neuroshapes.org")
context = forge.retrieve("https://bbp.neuroshapes.org")
context = forge.as_jsonld(context)["@context"]
context["label"] = {
      "@id": "rdfs:label",
      "@language": "en"
    }

context["prefLabel"] = {
      "@id": "skos:prefLabel",
      "@language": "en"
    }

context["altLabel"] = {
      "@id": "skos:altLabel",
      "@language": "en"
    }

context["definition"] = {
      "@id": "skos:definition",
      "@language": "en"
    }

In [ ]:
tag = "<IF ENABLE_TAG IS TRUE, PLEASE PROVIDE A TAG, E.G. THE REVISION NUMBER FROM WEBPROTEGE>"
bmo.register_ontology(forge, ontology_graph, context, ONTO_PATH, PREFIX, tag=None)

## Register individual classes

Create JSON payloads for individual classes

In [ ]:
bmo.remove_defines_relation(ontology_graph, PREFIX)

In [ ]:
class_jsons = bmo.frame_classes(ontology_graph, context, PREFIX)     

In [ ]:
class_jsons[0]

Register to Nexus

In [ ]:
bmo.register_classes(forge, class_jsons)